### 1. 데이터 생성하기
(1) ./datasets 아래에 .hwp 파일을 모두 load하기

(2) text_splitter로 분할하기

(3) all_docs에 모든 chunk들 append하기

(4) Pineone Vector database에 업로드하기

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_teddynote.document_loaders.hwp import HWPLoader
import os

document_list = []

# text splitter 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
)

# datasets 폴더를 시작점으로 하위 폴더까지 재귀적으로 순회
for root, dirs, files in os.walk("./datasets"):
    for file in files:
        if file.endswith(".hwp"):
            file_path = os.path.join(root, file)
            loader = HWPLoader(file_path)
            # 1) 문서 로드
            docs = loader.load()
            # 2) 텍스트 스플리터로 분할
            split_docs = text_splitter.split_documents(docs)
            document_list.extend(split_docs)

print(f"The number of Splitted Chunks: {len(document_list)}")

The number of Splitted Chunks: 3079


In [2]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [3]:
#Pinecone
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = 'ds-markdown'
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)

In [11]:
#Chroma
# from langchain_chroma import Chroma

# database=Chroma.from_documents(documents=document_list, embedding=embedding, collection_name='chroma-tax', persist_directory="./chroma")

#database=Chroma(collection_name='chroma-tax', persist_directory="./chroma", embedding_function=embedding) #이미 만들어 놓은 데베 사용

### 2. 답변 생성을 위한 Retrieval 및 Prompt 활용하기
(1) RetrievalQA에 전달하기 위해 retriever 생성

(2) LangChain에서 제공하는 rlm/rag-prompt 사용

In [4]:
query = '남해장학금 선발기준은?'
retriever = database.as_retriever(search_kwargs={'k': 4})
retriever.invoke(query)

[Document(id='3c0847d1-fd27-492d-86d1-505c397ff643', metadata={'source': './datasets/3편 학사행정/4장 교학행정/장학금규정시행세칙(20241113)_일부개정.hwp'}, page_content='성적이 일정기준 이상인 자로 단과대학별 모집인원의 상위 5% 이내인 자                                                                                                       | ⋅ 4년간(약학대학 6년) 수업료 전액면제<br>⋅ 학기 중 도서구입비 매월 30만원 지급<br>⋅ 해외자매대학에 교환유학생으로 1회 파견 또는 해외 어학 연수 1회 지원 (수업료+기숙사비+항공료 총 400만원 이내)<br>⋅ 지방학생의 경우 기숙사 우선선발 및 기숙사비 면제(1년)<br>⋅ 글로벌교육원 지정프로그램 무료수강<br>⋅ 대학일자리본부에서 운영하는 프로그램 우선 선발 || 남해장학금              | 각 단과대학별 수석합격자'),
 Document(id='aebed516-f02b-4f7e-ba90-9d9a4bbeca89', metadata={'source': './datasets/3편 학사행정/4장 교학행정/장학금규정시행세칙(20241113)_일부개정.hwp'}, page_content='|※ 기타 사항  가. 정시모집 장학생은 “가”군과 “나”군의 합격자를 통합하여 선발함. 단, 수시모집 미충원 이월에 따른 정시모집 정원외 전형은 제외<개정 2017.10.11., 2018.10.10>  나. 차미리사장학생의 수능 성적 기준<개정 2017.12.13., 2019.11.6., 2021.11.3..>      ① 글로벌융합대학, Art & Design대학 : 국어 1등급, 영어 1등급  ② 과학

In [5]:
from langchain import hub
from langchain_openai import ChatOpenAI

prompt = hub.pull("rlm/rag-prompt")
llm = ChatOpenAI(model='gpt-4o')

### 3. 답변 생성하기
(1) RetrievalQA를 통해 LLM에 전달

In [6]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, 
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt}
)

ai_message = qa_chain.invoke({"query": query})

ai_message

{'query': '남해장학금 선발기준은?',
 'result': '남해장학금은 각 단과대학별 수석합격자에게 수여됩니다. 수석합격자는 모집인원의 상위 5% 이내에 들어야 합니다.'}